In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

17334272/17329808 [==============================] - 65s 4us/step


In [3]:
import os
import pathlib

data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

Number of directories: 20
Directory names: ['talk.politics.mideast', 'rec.autos', 'comp.sys.mac.hardware', 'alt.atheism', 'rec.sport.baseball', 'comp.os.ms-windows.misc', 'rec.sport.hockey', 'sci.crypt', 'sci.med', 'talk.politics.misc', 'rec.motorcycles', 'comp.windows.x', 'comp.graphics', 'comp.sys.ibm.pc.hardware', 'sci.electronics', 'talk.politics.guns', 'sci.space', 'soc.religion.christian', 'misc.forsale', 'talk.religion.misc']


In [4]:
fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in comp.graphics: 1000
Some example filenames: ['38254', '38402', '38630', '38865', '38891']


In [5]:
print(open(data_dir / "comp.graphics" / "38987").read())


Newsgroups: comp.graphics
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!agate!dog.ee.lbl.gov!network.ucsd.edu!usc!rpi!nason110.its.rpi.edu!mabusj
From: mabusj@nason110.its.rpi.edu (Jasen M. Mabus)
Subject: Looking for Brain in CAD
Message-ID: <c285m+p@rpi.edu>
Nntp-Posting-Host: nason110.its.rpi.edu
Reply-To: mabusj@rpi.edu
Organization: Rensselaer Polytechnic Institute, Troy, NY.
Date: Thu, 29 Apr 1993 23:27:20 GMT
Lines: 7

Jasen Mabus
RPI student

	I am looking for a hman brain in any CAD (.dxf,.cad,.iges,.cgm,etc.) or picture (.gif,.jpg,.ras,etc.) format for an animation demonstration. If any has or knows of a location please reply by e-mail to mabusj@rpi.edu.

Thank you in advance,
Jasen Mabus  



In [6]:
import re
import string

def clean_text(text):
    text = text.lower()
    #text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('[''’“”_]', '', text)
    text = re.sub('\d', '', text)
    return text

In [7]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Precessing {}, {} files found".format(dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        content = clean_text(content)
        samples.append(content)
        labels.append(class_index)
    class_index += 1
    
print("Classes:", class_names)
print("Number of samples:", len(samples))

Precessing alt.atheism, 1000 files found
Precessing comp.graphics, 1000 files found
Precessing comp.os.ms-windows.misc, 1000 files found
Precessing comp.sys.ibm.pc.hardware, 1000 files found
Precessing comp.sys.mac.hardware, 1000 files found
Precessing comp.windows.x, 1000 files found
Precessing misc.forsale, 1000 files found
Precessing rec.autos, 1000 files found
Precessing rec.motorcycles, 1000 files found
Precessing rec.sport.baseball, 1000 files found
Precessing rec.sport.hockey, 1000 files found
Precessing sci.crypt, 1000 files found
Precessing sci.electronics, 1000 files found
Precessing sci.med, 1000 files found
Precessing sci.space, 1000 files found
Precessing soc.religion.christian, 997 files found
Precessing talk.politics.guns, 1000 files found
Precessing talk.politics.mideast, 1000 files found
Precessing talk.politics.misc, 1000 files found
Precessing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

In [8]:
seed = 123
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

In [9]:
train_split = 0.85
num_train_samples = int(train_split * len(samples))
train_X, train_Y = samples[:num_train_samples], labels[:num_train_samples]
val_X, val_Y = samples[num_train_samples:], labels[num_train_samples:]
print("train_X {}, train_Y {}".format(len(train_X), len(train_Y)))
print("val_X {}, val_Y {}".format(len(val_X), len(val_Y)))

train_X 16997, train_Y 16997
val_X 3000, val_Y 3000


In [10]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_X).batch(128)
vectorizer.adapt(text_ds)

In [11]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [12]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]


array([   2, 3132, 1823,   18,    2, 5935])

In [13]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [14]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test] 

[2, 3132, 1823, 18, 2, 5935]

### Load pre-trained word embeddings GloVe

In [15]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2020-12-17 11:49:56--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu... 171.64.67.140
Connecting to nlp.stanford.edu|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-12-17 11:49:56--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-12-17 11:49:57--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu... 171.64.64.22
Connecting to downloads.cs.stanford.edu|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: 'glove.6B.zip'

glove.6B.zip        100%[===================>] 822.24M   668KB/s    in 22m 40s 

2020-12-17 12:12:38 (619 KB/s) - 'glove

In [18]:
path_to_glove_file = os.path.join(
    "./glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [19]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 18383 words (1617 misses)


In [20]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [21]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         2000200   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         64128     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)        

In [22]:
x_train = vectorizer(np.array([[s] for s in train_X])).numpy()
x_val = vectorizer(np.array([[s] for s in val_X])).numpy()

y_train = np.array(train_Y)
y_val = np.array(val_Y)

In [23]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
133/133 [==============================] - 23s 174ms/step - loss: 2.6501 - acc: 0.1397 - val_loss: 2.0397 - val_acc: 0.3137
Epoch 2/20
133/133 [==============================] - 23s 171ms/step - loss: 1.9054 - acc: 0.3281 - val_loss: 1.5310 - val_acc: 0.4693
Epoch 3/20
133/133 [==============================] - 24s 184ms/step - loss: 1.4639 - acc: 0.4826 - val_loss: 1.2354 - val_acc: 0.5760
Epoch 4/20
133/133 [==============================] - 24s 178ms/step - loss: 1.1568 - acc: 0.5975 - val_loss: 1.0807 - val_acc: 0.6283
Epoch 5/20
133/133 [==============================] - 27s 201ms/step - loss: 0.9739 - acc: 0.6604 - val_loss: 1.0142 - val_acc: 0.6520
Epoch 6/20
133/133 [==============================] - 28s 214ms/step - loss: 0.8306 - acc: 0.7050 - val_loss: 0.9346 - val_acc: 0.6787
Epoch 7/20
133/133 [==============================] - 29s 215ms/step - loss: 0.6841 - acc: 0.7566 - val_loss: 0.9382 - val_acc: 0.6907
Epoch 8/20
133/133 [==============================] - 2

In [26]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about good li"]]
)

class_names[np.argmax(probabilities[0])]

'sci.med'